## Dependencies

In [1]:
# Custom data handling methods
import prep_IO

In [2]:
conv = prep_IO.acct_info_locs[0][:-14]
hosts_dict = prep_IO.sorted_arc_data

# Setup
- __FIXED_CTR_CTRL__ Counter Control Register
- __FIXED_CTR0__ Instructions Retired
- __FIXED_CTR1__ Core Clock Cycles
- __FIXED_CTR2__ Reference Clock Cycles

In [3]:
recent, stop = '2020-04-17', '2020-04-10'
sample_n = 1

sample_date = conv + recent + '.txt'
sample_stop = conv + stop + '.txt'

In [4]:
def find_recent_hosts( d0, dn, hosts_dict ):
    e = d0 + 'T00:00:00'
    s = dn + 'T23:59:59'
    out_dict = {}
    
    for host,host_list in hosts_dict.items():
        
        for host_file in host_list:
            t = prep_IO.get_time( host_file[-13:-3] )
             
            if t[:7] == s[:7]:
                
                if host in out_dict:
                    out_dict[host].append( host_file )
                    
                else:
                    out_dict[ host ] = [ host_file ]
                    
    return out_dict

def find_fixed( host_file ):    
    lines = prep_IO.unzip_txt( host_file )
    return [ line for line in lines if 'FIXED_CTR' in line ]

def find_fixed_set( host_dict, lim=0 ):
    out_dict = { host_name:[] for host_name in host_dict.keys() }
    
    for host_name, file_list in host_dict.items():
        
        if lim > 0:
            for host_file in file_list:
                out_dict[ host_name ] += find_fixed( host_file )
        else:
            cut = file_list[ : lim ]
            for host_file in cut:
                out_dict[ host_name ] += find_fixed( host_file )
                
    return out_dict

In [5]:
def cpi_calc( chunk ):
    core = chunk.find('CTR1')
    inst = chunk.find('CTR0')
    base = 24
    
    return core / inst / base

def fill_cpi( data_dict ):
    return { key: cpi_calc( value ) for key,value in data_dict.items() }

def cpicore_dict ( ):
    return {}

## A. cpi variance within calendar day
- 1 set equal weights
- 1 set consider length of job

In [7]:
# collect all unique (host, s, e, jobid) from date.txt acct file
search_set_A = prep_IO.acct_file_to_searchset( full_txt=sample_date )

# select date, default='yesterday'
# opt: collect the most recent n days from selected date
sample_info_A = prep_IO.search_sample_n( search_set_A, sample_n )

# collect each host_file from search set
sample_hosts_A = { key: sample_info_A['Out']['Source'][i]
                     for i in range(len( sample_info_A['Out']['Source'] ))
                     for key in sample_info_A['Out'].keys()     }

# collect core/inst lines from each file
sample_data_A = find_fixed_set( sample_hosts_A, lim=sample_n )

# compute cpi for each () using core/inst lines
sample_cpi_A = fill_cpi( sample_data_A )

## B. cpi variance between hosts
- 1 set equal weights
- 1 set consider number of jobids tasked

In [6]:
# select date, default='yesterday'
# opt: collect the most recent n days from selected date
sample_hosts_B = find_recent_hosts( recent, stop, hosts_dict )

# collect core/inst lines from each host file labeled with range of n days
sample_data_B = find_fixed_set( sample_hosts_B, lim=sample_n )

# compute cpi for each host using core/inst lines
sample_cpi_B = fill_cpi( sample_data_B )